###### December 2019 - Roger Melko

Diagonalizing the Hamiltonian matrix for the transverse field Ising model to find the energy eigenvalues and eigenkets.   Calculate the groundstate magnetization.

We will use the same Hamiltonian convention as the QMC program:
$$
H = -J\sum_{\langle i j \rangle} \sigma^z_i \sigma^z_j - B \sum_i \sigma^x_i
$$
where ${\bf \sigma}_i$ are Pauli operators.  In this convention, the 1+1 CFT is at $h/J = 1$.

In [1]:
using LinearAlgebra

In [8]:
N = 6
Dim = 2^N

J = 1. #this is the exchange
B = 1. #this is the transverse field

Hamiltonian = zeros(Dim,Dim)   #This is your 2D Hamiltonian matrix

for Ket = 0:Dim-1  #Loop over Hilbert Space
    Diagonal = 0.
    for SpinIndex = 0:N-2  #Loop over spin index (base zero, stop one spin before the end of the chain)
        Spin1 = 2*((Ket>>SpinIndex)&1) - 1
        NextIndex = SpinIndex + 1
        Spin2 = 2*((Ket>>NextIndex)&1) - 1
        Diagonal = Diagonal - J*Spin1*Spin2 #spins are +1 and -1
    end
    Hamiltonian[Ket+1,Ket+1] = Diagonal
    
    for SpinIndex = 0:N-1
        bit = 2^SpinIndex   #The "label" of the bit to be flipped
        Bra = Ket ⊻ bit    #Binary XOR flips the bit
        Hamiltonian[Bra+1,Ket+1] = -B
    end
end
Hamiltonian = Hermitian(Hamiltonian);

In the Julia LinearAlgebra package, the eigen function finds eigenvalues and eigenvectors.  They are ordered; i.e. the groundstate energy corresponds to index 1

In [9]:
Diag = eigen(Hamiltonian);

In [10]:
GroundState = Diag.vectors[:, 1];  #this gives the groundstate eigenvector
Diag.values[1] / N

-1.2160383017597935

In [12]:
##### Calculate the groundstate magnetization <m^2> in the Z direction
magnetization = zeros(Dim)
abs_mag = zeros(Dim)
mag_squared = zeros(Dim)

SumSz = dropdims(sum(@. (2 * (((0:Dim-1) >> (0:N-1)') & 1) - 1); dims=2); dims=2)
AbsSumSz = abs.(SumSz)
SumSzSq = abs2.(SumSz)

magnetization = SumSz' * abs2.(Diag.vectors)
abs_mag = AbsSumSz' * abs2.(Diag.vectors)
mag_squared = SumSzSq' * abs2.(Diag.vectors)

(magnetization[1] / N), (abs_mag[1] / N), (mag_squared[1] / (N*N))

(3.0568140611345975e-14, 0.6062539621278403, 0.4824376202801941)

In [6]:
β = 20.
weights = exp.(-β * Diag.values)
Z = sum(weights)
E = dot(Diag.values, weights) / (N*Z)

-1.2160330882932453

In [14]:
# magnetization of thermal state
M = dot(weights, magnetization) / (N*Z)
M_abs = dot(weights, abs_mag) / (N*Z)
M2 = dot(weights, mag_squared) / (N*N*Z)

M, M_abs, M2

(3.056486568179482e-14, 0.6062678612242338, 0.4824533210158867)